# Reading csv, json and shape format datasets. Combining them into one central file

One of the goals of our project is to study urban environment of Zurich city and afterwards associate it with insurance data. For this purpose, we will be utilizing public datasets published on <a href="url">https://data.stadt-zuerich.ch/</a> website. Datasets we will be using contain, but are not limited to, public parks, schools, street lights, public WCs and etc. in Zurich. 

This notebook is used to load all relevant datasets found on https://data.stadt-zuerich.ch/ as well as the population per zip code data from https://opendata.swiss/en/dataset/bevoelkerung-pro-plz into a central file that will be used for the rest of the analysis.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from os import listdir

In [2]:
DATA_FOLDER = "./data/"
CSV_JSON_FOLDER = DATA_FOLDER + './csv_json_files/'
SHAPE_FOLDER = DATA_FOLDER + "shapeFiles/"

## Reading and loading the csv files

In [3]:
# create function that reads cleans and appends csv to the general dataframe
def csv_read_clean_append(csv_file, duplicate_check, group_on, new_column, append_target=None):
    
    # load csv into dataframe, drop any duplacte rows, count values for group_on series 
    temp_df = pd.read_csv(CSV_JSON_FOLDER + csv_file).drop_duplicates(duplicate_check)[group_on].value_counts().to_frame(new_column)
    
    if append_target is None:
        return temp_df
    else:
        return append_target.join(temp_df)

In [4]:
# start with addresses and number of hospitality companies
combined_df = csv_read_clean_append('adressen.csv', 'adresse', 'plz', 'addresses')
combined_df = csv_read_clean_append('gastwirtschaftsbetriebe_per_20171231.csv',\
                                    'Betriebsname', 'plz', 'hospitality_companies', combined_df)

We now insert the population per zip code. We have to do some manual touching up, as the population is a sum of number of women and men.

In [5]:
# read population per zip code file
population_df = pd.read_csv(CSV_JSON_FOLDER +'bevoelkerung_proplz.csv',delimiter=';')

# sum the men and women count per zip code to get population per zip code
population_df = population_df[population_df['typ']\
                                .isin(['w', 'm'])]\
                                .groupby('plz')['anzahl']\
                                .agg('sum')\
                                .to_frame('population')

# merge the population data to the rest
combined_df = pd.merge(combined_df, population_df, 'inner', left_index=True, right_index=True)
combined_df.head()

,addresses,hospitality_companies,population
8050,4134,153.0,34412
8049,3498,34.0,27853
8048,3492,95.0,33499
8032,3403,50.0,23866
8046,3016,31.0,29139


## Reading and loading the json files

In [6]:
# create function that reads cleans and appends json datasets to the general dataframe
def json_read_clean_append(json_file, duplicate_check, group_on, new_column, append_target=None):
    
    # first read json using geopandas
    temp_df = gpd.read_file(CSV_JSON_FOLDER + json_file)
    
    
    # load json into dataframe, drop any duplicate rows, count values for group_on series 
    temp_df = pd.DataFrame(temp_df).drop_duplicates(duplicate_check)[group_on].value_counts().to_frame(new_column)
    temp_df.index = pd.to_numeric(temp_df.index)
    
    if append_target is None:
        return temp_df
    else:
        return pd.merge(append_target, temp_df, 'left', left_index=True,right_index=True)

In [7]:
# loop through json files in data folder and add to general df with function above
for file in listdir(CSV_JSON_FOLDER):
    if file.endswith('.json'):
        combined_df = json_read_clean_append(file,'adresse', 'plz', file[:-5], combined_df)

## Reading and loading the shape files

Shape files contain spatial information in form of point coordinates (longitude and latitude). In order to identify to which postal code they belong, we will be using zips.shp file that contains polygons describing area of zip codes in Switzerland. Furthermore, we will utilize geopandas functionality to join points with zip codes. One should also pay special attention to coordinate sysem. In order to join correctly all shape files should agree on coordinate system. Therefore, we will take coordinate systems used in zips.shp file as default and convert any other one into it for consistency.

In [8]:
#read zips data
zips = gpd.read_file(SHAPE_FOLDER + "zips.shp")
zips.columns = ['UUID', 'OS_UUID', 'STATUS', 'INAEND', 'ZIP', 'ZUSZIFF','geometry']

#default coordinate system
COORDINATE_SYSTEM = zips.crs
print("used coordinate system: ", zips.crs)

used coordinate system:  {'init': 'epsg:4326'}


Now we define method to read shape files into dataframe, to remove duplicates and convert coordinates to default coordinate system.

In [9]:
#default coordinate system
COORDINATE_SYSTEM = {'init': 'epsg:4326'}

def read_shape_file(file_name):
    df = gpd.read_file(SHAPE_FOLDER + file_name)
    
    #drop duplicates
    G = df["geometry"].apply(lambda geom: geom.wkb)
    df = df.loc[G.drop_duplicates().index]
    
    #convert to default coordinate systen
    if (df.crs != COORDINATE_SYSTEM):
        df = df.to_crs(COORDINATE_SYSTEM)
    
    return df

Furthermore, we define method that joins points with corresponding polygons. Meaning we will much each coordinate (longitute and latitude) with corresponding zip code in Zurich and then groups counts by zip code.

In [10]:
def join_points_with_zips(df):
    """join coordinates with zip codes"""
    pointInPoly = gpd.sjoin(df, zips, op='within') 
    return pointInPoly

In [11]:
def grouping_by_zipcode(df, new_column):
    """group item counts per zip codes"""
    df = df.groupby('ZIP').size().reset_index(name=new_column)
    df.set_index('ZIP', inplace=True)
    return df

The method below will be used for appending information in the shape file to the existing central dataframe for aggregation of all information into one file.

In [12]:
# create function that reads cleans and appends shape datasets to the general dataframe
def shape_read_clean_append(shape_file, new_column, append_target=None):
    
    # first read shape, drop duplicates
    temp_df = read_shape_file(shape_file)
    
    temp_df = join_points_with_zips(temp_df)
    
    # load shape into dataframe, drop any duplicate rows, count values for group_on series 
    temp_df = grouping_by_zipcode(temp_df, new_column)
    temp_df.index = pd.to_numeric(temp_df.index)
    #display(temp_df.head(2))
    
    if append_target is None:
        return temp_df
    else:
        return pd.merge(append_target, temp_df, 'left', left_index=True,right_index=True)

Now we will read all shape files from corresponding folder and load them into the central dataframe.

In [13]:
# loop through shp files in data folder and add to general df with function above
for file in listdir(SHAPE_FOLDER):
    if file.endswith('.shp'):
        combined_df = shape_read_clean_append(file, file[:-4], combined_df)

("External Dataset Preprocessing.ipynb" contains some additional information about shape files)

## Final Merging

At the end we save combined information into .csv file for further usage.

In [14]:
# drop zip codes that are not used (last 5 rows), and fill other nans with zeros
#combined_df = combined_df.dropna(subset=['hospitality_companies']).fillna(0)
combined_df.to_csv(DATA_FOLDER + "open_data_aggregated.csv")
combined_df

,addresses,hospitality_companies,population,beachvolleyball,bikeparks,care_centers,community_centers,elementary_schools,football_fields,ice_rinks,...,Kirche,KunstImStadtraum,Park,Picknickplatz,Sozialzentrum,Sporthalle,Stadion,zips,ZueriWC_nichtrollstuhlgaengig,ZueriWC_rollstuhlgaengig
8050,4134,153.0,34412,NaN,NaN,1.0,1.0,8.0,2.0,1.0,...,2.0,15.0,16.0,1.0,1.0,3.0,NaN,1,1.0,5.0
8049,3498,34.0,27853,1.0,NaN,1.0,1.0,7.0,NaN,NaN,...,2.0,6.0,4.0,10.0,NaN,NaN,NaN,1,1.0,3.0
8048,3492,95.0,33499,3.0,NaN,1.0,1.0,9.0,3.0,NaN,...,4.0,10.0,1.0,6.0,NaN,NaN,1.0,1,NaN,3.0
8032,3403,50.0,23866,NaN,NaN,NaN,1.0,5.0,NaN,NaN,...,2.0,20.0,4.0,3.0,NaN,NaN,NaN,1,NaN,5.0
8046,3016,31.0,29139,NaN,NaN,1.0,1.0,7.0,1.0,NaN,...,3.0,4.0,7.0,3.0,NaN,2.0,NaN,1,1.0,1.0
8008,2929,110.0,23162,NaN,NaN,1.0,1.0,7.0,NaN,NaN,...,3.0,34.0,13.0,1.0,NaN,1.0,NaN,1,4.0,5.0
8006,2894,69.0,22387,1.0,NaN,NaN,1.0,5.0,NaN,NaN,...,5.0,22.0,6.0,5.0,NaN,NaN,NaN,1,NaN,4.0
8057,2786,35.0,24258,NaN,NaN,2.0,1.0,3.0,NaN,NaN,...,6.0,19.0,3.0,3.0,NaN,NaN,NaN,1,NaN,4.0
8051,2714,37.0,25792,1.0,NaN,1.0,1.0,9.0,1.0,NaN,...,4.0,4.0,4.0,5.0,NaN,2.0,NaN,1,NaN,3.0
8004,2663,359.0,34271,NaN,NaN,NaN,NaN,6.0,NaN,NaN,...,5.0,19.0,6.0,NaN,NaN,1.0,NaN,1,4.0,7.0


As we see datasets we will be using consist of:

- Addresses - 57923 in total
- Hospitality companies - 2097 in total
- Population - 509285 in total
- Beach volleyball - 12 in total
- Bike Parks - 1 in total
- Care centers - 20 in total
- Community centers - 18 in total
- Elementary schools - 115 in total
- Football fields - 15 in total
- Ice rinks - 3 in total
- Indoor pools - 7 in total
- Kindergartens - 241 in total
- Mobility rental - 227 in total
- Nurseries - 316 in total
- Outdoor pools - 6 in total
- Police locations - 28 in total
- Skate parks - 8 in total
- Tennis courts - 8 in total
- Parks - 118 in total
- Street Lights (Beleuchtung) - 40010 in total
- Old people center (Alterszentrum) - 28 in total
- Retirment houses (Alterswohnung) - 36 in total
- Handicapped parking (Behindertenparkplatz) - 410 in total
- Fontaines (brunnen) - 1281 in total
- Youth clubs/centers/ meeting center (Jugendtreff) - 13 in total
- Kindergarten - 354 in total
- Kinderhaus / Eltern-Kind-Zentrum - 16 in total
- Churches (Kirche) - 82 in total
- Temporary art in the urban space (Kunst im Stadtraum) - 388 in total
- picknickplatz - 110 in total
- social center (sozial zentrum) - 5 in total
- Gym (Sporthalle) - 16 in total
- Stadium - 1 in total (therefore won't be using)
- WC handicapped - 28 in total
- WC not handicapped - 77 in total